<a href="https://colab.research.google.com/github/massaoiwanaga/topespsi/blob/main/TESI_VAC_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Trabalho Disciplina Topicos Especiais em Sistemas de Informação I**

---

## **Saúde (COVID-19)**

> Alessandra Pierro Saraiva
>
> Fabiane Monteiro Carvalho
>
> Massao Massao Oliveira Iwanaga

### Importando a biblioteca pandas

In [ ]:
# importando a biblioteca
import pandas as pd # pandas
import numpy as np  # numpy

### Instalação da biblioteca PROV

In [ ]:
#Instalação da biblioteca PROV no ambiente COLAB
#Essa biblioteca implementa em Python o Modelo de Dados PROV da W3C
!pip install prov

### Acessar dados do Google Colab

In [ ]:
# Se você está executando o notebook no Google Colab, é necessário montar o drive para o ambiente virtual do Google Drive
# Clicar no link que aparecerá e será gerada uma chave, que deverá ser colada no campo "Autorization code"

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Salve o dataset de vacinação no diretório TESI_FCD no Google Drive
# Para tornar o notebook reutilizável, futuramente pode-se solicitar o diretório e o nome do dataset via prompt.

df_vacinacao = pd.read_csv('/content/drive/My Drive/TESI_FCD/part-00000-090405dc-80c4-4889-84c3-a9a390d06947-c000.csv',delimiter=',')

### Exploração inicial do DataSet

In [ ]:
print('Total de registros:',len(df_vacinacao))    # Quantidade de registros do dataset
print('')
print(df_vacinacao.info())  # Informações sobre o dataset 

Total de registros: 6958578

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6958578 entries, 0 to 6958577
Data columns (total 33 columns):
 #   Column                                    Dtype  
---  ------                                    -----  
 0   document_id                               object 
 1   paciente_id                               object 
 2   paciente_idade                            float64
 3   paciente_dataNascimento                   object 
 4   paciente_enumSexoBiologico                object 
 5   paciente_racaCor_codigo                   float64
 6   paciente_racaCor_valor                    object 
 7   paciente_endereco_coIbgeMunicipio         object 
 8   paciente_endereco_coPais                  object 
 9   paciente_endereco_nmMunicipio             object 
 10  paciente_endereco_nmPais                  object 
 11  paciente_endereco_uf                      object 
 12  paciente_endereco_cep                     object 
 13  paciente_nacionalidade_enumN

In [ ]:
df_vacinacao.head(10)        # Visualização dos registros iniciais

,document_id,paciente_id,paciente_idade,paciente_dataNascimento,paciente_enumSexoBiologico,paciente_racaCor_codigo,paciente_racaCor_valor,paciente_endereco_coIbgeMunicipio,paciente_endereco_coPais,paciente_endereco_nmMunicipio,paciente_endereco_nmPais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumNacionalidade,estabelecimento_valor,estabelecimento_razaoSocial,estalecimento_noFantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoAtendimento_codigo,vacina_grupoAtendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataAplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds
0,3b970408-1a74-4089-8334-e18a0d1f4398-i0b0,051213f3f26a8fac4acd490ff737c6695e67a2ade7aeef...,36.0,1984-05-07,F,1.0,BRANCA,430510,10,CAXIAS DO SUL,BRASIL,RS,95020,B,2223597,HOSPITAL SAUDE LTDA,HOSPITAL SAUDE,430510.0,CAXIAS DO SUL,RS,907,Enfermeiro(a),9.0,Trabalhadores de Saúde,210010,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-11T03:00:00.000Z,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-12 15:33:19
1,c6756cf9-0e0e-456b-86d2-94309b47e702-i0b0,3c73310d030b31ec5c73deda79bad2e8dbcc388839b6cd...,43.0,1977-12-05,F,1.0,BRANCA,352270,10,ITAPOLIS,BRASIL,SP,14900,B,2747723,PREFEITURA MUNICIPAL DE ITAPOLIS,UBS JARDIM REDENCAO ITAPOLIS,352270.0,ITAPOLIS,SP,926,Outros,9.0,Trabalhadores de Saúde,202010024,Sinovac,NaN,2021-02-12T00:00:00.000Z,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,VACIVIDA,2021-02-12 10:02:44
2,bac70bd9-608b-4a3b-8136-fa4b26fa0e10-i0b0,4278472f0c1d82ad758d6312ca67b17e296e82060d4fb4...,67.0,1953-04-25,M,99.0,SEM INFORMACAO,510320,10,COLIDER,BRASIL,MT,78500,B,2534355,PREFEITURA MUNICIPAL DE CUIABA,USF ALVORADA,510340.0,CUIABA,MT,912,Médico,9.0,Trabalhadores de Saúde,4120Z005,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-02-03T03:00:00.000Z,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-02-03 16:06:04
3,c6143b69-6e49-4db0-94db-8e0acdd03888-i0b0,4dbc4710b67953ae5f3969eead5d70e3821574dd39f605...,65.0,1955-03-05,M,4.0,AMARELA,290680,10,CANSANCAO,BRASIL,BA,48840,B,9502688,PREFEITURA MUNICIPAL DE CANSANCAO,UNIDADE DE SAUDE DA FAMILIA LAURA SALVADOR,290680.0,CANSANCAO,BA,926,Outros,9.0,Trabalhadores de Saúde,202009014,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-17T03:00:00.000Z,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-17 18:22:19
4,4be2fede-1871-4447-a9e3-0e55c324836e-i0b0,a1c51574bd53ef0d66cae08f8329f7584e1f3c06ca992e...,35.0,1986-01-08,M,99.0,SEM INFORMACAO,260600,10,GARANHUNS,BRASIL,PE,"\\""""",B,3002586,MUNICIPIO DE GARANHUNS,USF DA LIBERDADE,260600.0,GARANHUNS,PE,926,Outros,9.0,Trabalhadores de Saúde,202010041,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-12T03:00:00.000Z,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-18 20:45:02
5,cb301c9e-5f42-4eec-b96f-4e033083e6b6-i0b0,bc8596e3c64a543e73c4d9d3999334bfbe810e155f6b52...,55.0,1965-10-10,M,3.0,PARDA,521630,10,PARANAIGUARA,BRASIL,GO,75880,B,5343577,PREFEITURA MUNICIPAL DE PARANAIGUARA,ESTRATEGIA SAUDE DA FAMILIA 2,521630.0,PARANAIGUARA,GO,914,Motorista de Ambulância,9.0,Trabalhadores de Saúde,202010031,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-19T03:00:00.000Z,2ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-19 17:53:47
6,8f4eb329-7faf-4c69-a61c-9dc101542879-i0b0,db1db6b5eb18d38b0e40a112ba515a8f36adb47d605276...,25.0,1995-11-19,F,99.0,SEM INFORMACAO,130410,10,TAPAUA,BRASIL,AM,69480,B,7723628,MINISTERIO DA SAUDE,DISTRITO SANITARIO ESPECIAL INDIGENA MEDIO PURUS,130240.0,LABREA,AM,701,Povos indígenas em terras indígenas,7.0,Povos Indígenas,202010028,FUNDACAO BUTANTAN,Organization/61189445000156,2021-02-03T03:00:00.000Z,1ª Dose,86,Covid-19-Coronavac-Sinovac/Butantan,Novo PNI,2021-02-11 19:06:14
7,b555c0f9-3883-4446-bfbd-35cd15d62a7d-i0b0,4a86688c8038ed2d38e1fd536ec6666b7eaefaa8c7efbd...,77.0,

In [ ]:
df_vacinacao.count()        # Quantidade de registros por coluna. 
                      # É possível observar que algumas colunas possuem quatidade inferior de registros, o que indica que será necessário 
                      #    tratamento de dados se essas colunas forem necessárias para análise

document_id                                 6958578
paciente_id                                 6958577
paciente_idade                              6958577
paciente_dataNascimento                     6958577
paciente_enumSexoBiologico                  6958577
paciente_racaCor_codigo                     6958577
paciente_racaCor_valor                      6958577
paciente_endereco_coIbgeMunicipio           6958578
paciente_endereco_coPais                    6958578
paciente_endereco_nmMunicipio               6958578
paciente_endereco_nmPais                    6958578
paciente_endereco_uf                        6958578
paciente_endereco_cep                       6958578
paciente_nacionalidade_enumNacionalidade    6947142
estabelecimento_valor                       6958578
estabelecimento_razaoSocial                 6958451
estalecimento_noFantasia                    6958451
estabelecimento_municipio_codigo            6958451
estabelecimento_municipio_nome              6958451
estabelecime